In [15]:
dictfile = "/Users/joregan/Documents/MFA/pretrained_models/dictionary/hsi_english_us_arpa.dict"
tgfiles = "/Users/joregan/Playing/hsi_ctmedit/textgrid/"
outdict = "/tmp/added.dict"

In [4]:
EXP = {
    "JH": "Y",
    "W": "V",
    "V": "B",
    "B": "V"
}

TRAP_BATH = ["S", "F", "TH", "N", "V", "Z", "SH", "M", "L", "DH"]

def expand_inner(text):
    phones = text.split(" ")
    outphones = []
    if len(phones) >= 2 and phones[0] == "S" and phones[1] in ["T", "P", "M"]:
        outphones.append(["EH0", ""])
    i = 0
    next_opt = False
    for i in range(len(phones)):
        cur = [phones[i]]
        if next_opt:
            cur.append("")
            next_opt = False
        if phones[i] in EXP:
            cur.append(EXP[phones[i]])
        if i < len(phones) - 1 and phones[i] in ["S", "N"] and phones[i+1] == "T":
            next_opt = True
        if i < len(phones) - 1 and phones[i].startswith("AE") and phones[i+1] in TRAP_BATH:
            cur.append("AA" + phones[i][-1])
        if i < len(phones) - 1 and phones[i].startswith("AA") and phones[i+1] == "R":
            cur.append("AE" + phones[i][-1])

        outphones.append(cur)
    return outphones            


In [10]:
import itertools

def expand(text):
    return [list(x) for x in expand_inner(text)]


In [13]:
entries = {}
with open(dictfile) as df:
    for line in df.readlines():
        parts = line.split("\t")
        if not parts[0] in entries:
            entries[parts[0]] = []
        entries[parts[0]].append(parts[-1])

In [16]:
from pathlib import Path
from praatio import textgrid
import re

def norm(text):
    words = text.split(" ")
    words = [w.strip("\",.;:?!").upper() for w in words if not w.startswith("[")]
    return words


seen_words = []
missing = []
new_entries = set()

for textgridfile in Path(tgfiles).glob("*.[Tt]ext[Gg]rid"):
    tg = textgrid.openTextgrid(textgridfile, includeEmptyIntervals=False)

    if len(tg.tierNames) == 1:
        tier = tg.getTier(tg.tierNames[0])
    elif "whisperx" in tg.tierNames:
        tier = tg.getTier("whisperx")
    elif "utterances" in tg.tierNames:
        tier = tg.getTier("utterances")
    elif "words" in tg.tierNames:
        tier = tg.getTier("words")
    else:
        print("Be careful: file", textgridfile, "has none of the expected tier names")

    for interval in tier.entries:
        start = interval[0]
        end = interval[1]
        text = interval[2]

        m = re.match("^\[[^]]+\]$", text)
        if m:
            continue
        
        for word in norm(text):
            if word in seen_words:
                continue
            if not word in entries:
                missing.append(word)
                continue
            for pron in entries[word]:
                for expanded in expand(pron):
                    # filter existing pronunciations!
                    new_entries.add(f"{word}\t{' '.join(expanded)}")